# File Location: docs/notebooks/08_testing_strategies.ipynb

# Testing Strategies in Python - Interactive Learning Notebook

Welcome to Testing Strategies in Python! This notebook covers comprehensive testing methodologies, frameworks, and best practices for building reliable Python applications.

## Learning Objectives

After completing this notebook, you will be able to:

- Understand different types of testing and their purposes
- Write effective unit tests with pytest
- Implement integration and end-to-end testing
- Use mocking and test doubles effectively
- Apply test-driven development (TDD) principles
- Measure and improve test coverage
- Set up continuous integration testing
- Debug and troubleshoot test failures

## Table of Contents

1. [Testing Fundamentals](#testing-fundamentals)
2. [Unit Testing with pytest](#unit-testing)
3. [Test Doubles and Mocking](#test-doubles)
4. [Integration Testing](#integration-testing)
5. [Test-Driven Development](#test-driven-development)
6. [Test Coverage and Quality](#test-coverage)
7. [Performance Testing](#performance-testing)
8. [Continuous Integration](#continuous-integration)
9. [Testing Best Practices](#best-practices)
10. [Advanced Testing Patterns](#advanced-patterns)

---

## 1. Testing Fundamentals

### The Testing Pyramid

```python
"""
Testing Strategy Overview:

                    ┌─────────────────┐
                    │   E2E Tests     │  ← Few, slow, high confidence
                    │   (UI/API)      │
                    └─────────────────┘
                           │
               ┌─────────────────────────┐
               │  Integration Tests      │  ← Some, medium speed
               │  (Component/Service)    │
               └─────────────────────────┘
                           │
        ┌─────────────────────────────────────┐
        │      Unit Tests                     │  ← Many, fast, isolated
        │   (Functions/Classes)               │
        └─────────────────────────────────────┘

Test Types by Purpose:
1. Unit Tests: Test individual components in isolation
2. Integration Tests: Test component interactions
3. End-to-End Tests: Test complete user workflows
4. Performance Tests: Test system performance under load
5. Security Tests: Test for vulnerabilities
6. Acceptance Tests: Test business requirements
"""

class TestingFramework:
    """Conceptual framework for understanding testing"""
    
    def __init__(self):
        self.test_types = {
            "unit": {
                "purpose": "Test individual functions/classes in isolation",
                "scope": "Single unit of code",
                "speed": "Fast (milliseconds)",
                "cost": "Low",
                "feedback": "Immediate",
                "examples": ["Function returns correct value", "Class method behaves as expected"]
            },
            "integration": {
                "purpose": "Test interaction between components",
                "scope": "Multiple units working together",
                "speed": "Medium (seconds)",
                "cost": "Medium",
                "feedback": "Quick",
                "examples": ["Database integration", "API calls", "File I/O"]
            },
            "end_to_end": {
                "purpose": "Test complete user workflows",
                "scope": "Entire application",
                "speed": "Slow (minutes)",
                "cost": "High",
                "feedback": "Delayed",
                "examples": ["User registration flow", "Purchase process", "Report generation"]
            },
            "performance": {
                "purpose": "Test system performance and scalability",
                "scope": "System under load",
                "speed": "Slow (minutes to hours)",
                "cost": "High",
                "feedback": "Detailed metrics",
                "examples": ["Load testing", "Stress testing", "Memory usage"]
            }
        }
    
    def describe_test_type(self, test_type):
        if test_type in self.test_types:
            details = self.test_types[test_type]
            print(f"{test_type.upper()} TESTS:")
            for key, value in details.items():
                print(f"  {key.title()}: {value}")
        else:
            print(f"Unknown test type: {test_type}")
    
    def show_testing_strategy(self):
        print("Comprehensive Testing Strategy:")
        print("=" * 35)
        
        for test_type in ["unit", "integration", "end_to_end", "performance"]:
            self.describe_test_type(test_type)
            print()

# Demonstrate testing concepts
framework = TestingFramework()
framework.show_testing_strategy()
```

---

## 2. Unit Testing with pytest

### Basic Calculator Example

```python
# Example of a simple module to test
class Calculator:
    """Simple calculator class for demonstration"""
    
    def add(self, a, b):
        """Add two numbers"""
        return a + b
    
    def subtract(self, a, b):
        """Subtract second number from first"""
        return a - b
    
    def multiply(self, a, b):
        """Multiply two numbers"""
        return a * b
    
    def divide(self, a, b):
        """Divide first number by second"""
        if b == 0:
            raise ValueError("Cannot divide by zero")
        return a / b
    
    def power(self, base, exponent):
        """Raise base to the power of exponent"""
        if exponent < 0:
            raise ValueError("Negative exponents not supported")
        return base ** exponent

# Test class demonstrating pytest patterns
class TestCalculator:
    """Test cases for Calculator class"""
    
    def setup_method(self):
        """Set up test fixtures before each test method"""
        self.calc = Calculator()
    
    def test_add_positive_numbers(self):
        """Test addition of positive numbers"""
        # Arrange
        a, b = 5, 3
        expected = 8
        
        # Act
        result = self.calc.add(a, b)
        
        # Assert
        assert result == expected
    
    def test_divide_by_zero_raises_error(self):
        """Test that division by zero raises ValueError"""
        try:
            self.calc.divide(10, 0)
            assert False, "Expected ValueError was not raised"
        except ValueError as e:
            assert str(e) == "Cannot divide by zero"

def run_calculator_tests():
    """Simulate running the calculator tests"""
    print("Running Calculator Tests:")
    print("=" * 27)
    
    test_instance = TestCalculator()
    test_methods = [method for method in dir(test_instance) if method.startswith('test_')]
    
    passed = 0
    failed = 0
    
    for test_method in test_methods:
        try:
            test_instance.setup_method()
            getattr(test_instance, test_method)()
            print(f"✓ {test_method}")
            passed += 1
        except Exception as e:
            print(f"✗ {test_method}: {e}")
            failed += 1
    
    print(f"\nResults: {passed} passed, {failed} failed")

run_calculator_tests()
```

---

## 3. Test Doubles and Mocking

```python
# Mock objects and test doubles
class MockEmailService:
    """Mock email service for testing"""
    
    def __init__(self):
        self.sent_emails = []
        self.should_fail = False
    
    def send_email(self, to, subject, body):
        """Mock send email method"""
        if self.should_fail:
            raise Exception("Email service unavailable")
        
        email = {
            'to': to,
            'subject': subject,
            'body': body,
            'timestamp': '2023-01-01 12:00:00'
        }
        self.sent_emails.append(email)
        return True
    
    def get_sent_count(self):
        return len(self.sent_emails)

class UserRegistrationService:
    """Service that depends on email service"""
    
    def __init__(self, email_service):
        self.email_service = email_service
        self.users = {}
    
    def register_user(self, username, email, password):
        """Register a new user and send welcome email"""
        if username in self.users:
            raise ValueError("Username already exists")
        
        if not email or '@' not in email:
            raise ValueError("Invalid email address")
        
        if len(password) < 6:
            raise ValueError("Password must be at least 6 characters")
        
        # Create user
        user = {
            'username': username,
            'email': email,
            'password': password,  # In real app, this would be hashed
            'active': True
        }
        self.users[username] = user
        
        # Send welcome email
        subject = f"Welcome {username}!"
        body = f"Thank you for registering with us, {username}."
        self.email_service.send_email(email, subject, body)
        
        return user

class TestUserRegistrationService:
    """Tests for UserRegistrationService using mocks"""
    
    def setup_method(self):
        """Set up test fixtures"""
        self.mock_email = MockEmailService()
        self.service = UserRegistrationService(self.mock_email)
    
    def test_successful_registration_sends_email(self):
        """Test that successful registration sends welcome email"""
        # Arrange
        username = "testuser"
        email = "test@example.com"
        password = "password123"
        
        # Act
        user = self.service.register_user(username, email, password)
        
        # Assert
        assert user['username'] == username
        assert user['email'] == email
        assert user['active'] is True
        
        # Verify email was sent
        assert self.mock_email.get_sent_count() == 1
        sent_email = self.mock_email.sent_emails[0]
        assert sent_email['to'] == email
        assert username in sent_email['subject']
    
    def test_registration_with_duplicate_username_fails(self):
        """Test that duplicate username registration fails"""
        # Arrange
        username = "duplicate"
        
        # Act & Assert
        self.service.register_user(username, "test1@example.com", "password123")
        
        try:
            self.service.register_user(username, "test2@example.com", "password456")
            assert False, "Expected ValueError for duplicate username"
        except ValueError as e:
            assert "already exists" in str(e)
        
        # Verify only one email was sent
        assert self.mock_email.get_sent_count() == 1
    
    def test_registration_continues_even_if_email_fails(self):
        """Test that user registration succeeds even if email fails"""
        # Arrange
        self.mock_email.should_fail = True
        
        # Act & Assert
        try:
            self.service.register_user("testuser", "test@example.com", "password123")
            assert False, "Expected exception due to email failure"
        except Exception as e:
            assert "Email service unavailable" in str(e)

def demonstrate_mocking():
    """Demonstrate mocking concepts"""
    print("\nMocking and Test Doubles Demo:")
    print("=" * 32)
    
    # Run the tests
    test_instance = TestUserRegistrationService()
    test_methods = [m for m in dir(test_instance) if m.startswith('test_')]
    
    for test_method in test_methods:
        try:
            test_instance.setup_method()
            getattr(test_instance, test_method)()
            print(f"✓ {test_method}")
        except Exception as e:
            print(f"✗ {test_method}: {e}")

demonstrate_mocking()
```

---

## 4. Integration Testing

```python
# Simulate a simple database and API for integration testing
class SimpleDatabase:
    """Simple in-memory database for testing"""
    
    def __init__(self):
        self.tables = {}
        self.connected = False
    
    def connect(self):
        """Connect to database"""
        self.connected = True
        return True
    
    def disconnect(self):
        """Disconnect from database"""
        self.connected = False
    
    def create_table(self, table_name, schema):
        """Create a table with schema"""
        if not self.connected:
            raise Exception("Database not connected")
        
        self.tables[table_name] = {
            'schema': schema,
            'data': [],
            'auto_increment': 1
        }
    
    def insert(self, table_name, data):
        """Insert data into table"""
        if not self.connected:
            raise Exception("Database not connected")
        
        if table_name not in self.tables:
            raise Exception(f"Table {table_name} does not exist")
        
        # Add auto-increment ID
        record = data.copy()
        record['id'] = self.tables[table_name]['auto_increment']
        self.tables[table_name]['auto_increment'] += 1
        
        self.tables[table_name]['data'].append(record)
        return record['id']
    
    def select(self, table_name, where=None):
        """Select data from table"""
        if not self.connected:
            raise Exception("Database not connected")
        
        if table_name not in self.tables:
            return []
        
        data = self.tables[table_name]['data']
        
        if where is None:
            return data.copy()
        
        # Simple where clause filtering
        results = []
        for record in data:
            match = True
            for key, value in where.items():
                if record.get(key) != value:
                    match = False
                    break
            if match:
                results.append(record)
        
        return results

class UserRepository:
    """Repository pattern for user data access"""
    
    def __init__(self, database):
        self.db = database
        self._ensure_table_exists()
    
    def _ensure_table_exists(self):
        """Ensure users table exists"""
        if 'users' not in self.db.tables:
            schema = {
                'id': 'INTEGER PRIMARY KEY',
                'username': 'VARCHAR(50) UNIQUE',
                'email': 'VARCHAR(100)',
                'created_at': 'TIMESTAMP'
            }
            self.db.create_table('users', schema)
    
    def create_user(self, username, email):
        """Create a new user"""
        from datetime import datetime
        
        # Check if username exists
        existing = self.db.select('users', {'username': username})
        if existing:
            raise ValueError("Username already exists")
        
        user_data = {
            'username': username,
            'email': email,
            'created_at': datetime.now().isoformat()
        }
        
        user_id = self.db.insert('users', user_data)
        return self.get_user_by_id(user_id)
    
    def get_user_by_id(self, user_id):
        """Get user by ID"""
        users = self.db.select('users', {'id': user_id})
        return users[0] if users else None
    
    def get_user_by_username(self, username):
        """Get user by username"""
        users = self.db.select('users', {'username': username})
        return users[0] if users else None
    
    def get_all_users(self):
        """Get all users"""
        return self.db.select('users')

class TestUserRepositoryIntegration:
    """Integration tests for UserRepository and Database"""
    
    def setup_method(self):
        """Set up test database"""
        self.db = SimpleDatabase()
        self.db.connect()
        self.user_repo = UserRepository(self.db)
    
    def teardown_method(self):
        """Clean up test database"""
        self.db.disconnect()
    
    def test_create_and_retrieve_user(self):
        """Test creating and retrieving a user"""
        # Arrange
        username = "testuser"
        email = "test@example.com"
        
        # Act
        created_user = self.user_repo.create_user(username, email)
        retrieved_user = self.user_repo.get_user_by_username(username)
        
        # Assert
        assert created_user is not None
        assert created_user['username'] == username
        assert created_user['email'] == email
        assert 'id' in created_user
        assert 'created_at' in created_user
        
        assert retrieved_user == created_user
    
    def test_duplicate_username_raises_error(self):
        """Test that duplicate usernames are not allowed"""
        # Arrange
        username = "duplicate"
        
        # Act & Assert
        self.user_repo.create_user(username, "user1@example.com")
        
        try:
            self.user_repo.create_user(username, "user2@example.com")
            assert False, "Expected ValueError for duplicate username"
        except ValueError as e:
            assert "already exists" in str(e)
    
    def test_get_all_users(self):
        """Test retrieving all users"""
        # Arrange
        users_to_create = [
            ("user1", "user1@example.com"),
            ("user2", "user2@example.com"),
            ("user3", "user3@example.com")
        ]
        
        # Act
        for username, email in users_to_create:
            self.user_repo.create_user(username, email)
        
        all_users = self.user_repo.get_all_users()
        
        # Assert
        assert len(all_users) == 3
        usernames = [user['username'] for user in all_users]
        assert "user1" in usernames
        assert "user2" in usernames
        assert "user3" in usernames

def run_integration_tests():
    """Run integration tests"""
    print("\nIntegration Tests Demo:")
    print("=" * 24)
    
    test_instance = TestUserRepositoryIntegration()
    test_methods = [m for m in dir(test_instance) if m.startswith('test_')]
    
    passed = 0
    failed = 0
    
    for test_method in test_methods:
        try:
            test_instance.setup_method()
            getattr(test_instance, test_method)()
            test_instance.teardown_method()
            print(f"✓ {test_method}")
            passed += 1
        except Exception as e:
            print(f"✗ {test_method}: {e}")
            failed += 1
    
    print(f"\nIntegration Results: {passed} passed, {failed} failed")

run_integration_tests()
```

---

## 5. Test-Driven Development (TDD)

```python
"""
Test-Driven Development (TDD) Demo

TDD Cycle:
1. RED: Write a failing test
2. GREEN: Write minimal code to make it pass
3. REFACTOR: Improve the code while keeping tests green

Example: Building a Shopping Cart
"""

class TestShoppingCart:
    """TDD Example: Building a shopping cart step by step"""
    
    def setup_method(self):
        """Set up test fixtures"""
        # This will be implemented as we develop ShoppingCart
        pass
    
    # STEP 1: RED - Write first failing test
    def test_new_cart_is_empty(self):
        """Test that a new shopping cart is empty"""
        cart = ShoppingCart()
        assert cart.is_empty() is True
        assert cart.total_items() == 0
    
    # STEP 2: More tests to drive development
    def test_add_item_to_cart(self):
        """Test adding an item to the cart"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.50, 2)
        
        assert cart.is_empty() is False
        assert cart.total_items() == 2
        assert cart.total_cost() == 3.00
    
    def test_add_multiple_different_items(self):
        """Test adding multiple different items"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.50, 2)
        cart.add_item("Banana", 0.75, 3)
        
        assert cart.total_items() == 5
        assert cart.total_cost() == 5.25
    
    def test_add_same_item_multiple_times(self):
        """Test adding the same item multiple times"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.50, 2)
        cart.add_item("Apple", 1.50, 1)
        
        items = cart.get_items()
        assert len(items) == 1  # Should combine items
        assert items["Apple"]["quantity"] == 3
        assert cart.total_cost() == 4.50
    
    def test_remove_item_from_cart(self):
        """Test removing items from cart"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.50, 3)
        cart.remove_item("Apple", 1)
        
        items = cart.get_items()
        assert items["Apple"]["quantity"] == 2
        assert cart.total_cost() == 3.00
    
    def test_remove_all_of_item(self):
        """Test removing all quantity of an item"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.50, 2)
        cart.remove_item("Apple", 2)
        
        assert cart.is_empty() is True
        assert "Apple" not in cart.get_items()
    
    def test_apply_discount(self):
        """Test applying discount to cart"""
        cart = ShoppingCart()
        cart.add_item("Apple", 1.00, 10)
        cart.apply_discount(0.1)  # 10% discount
        
        assert cart.total_cost() == 9.00

# GREEN: Implement minimal ShoppingCart to make tests pass
class ShoppingCart:
    """Shopping cart implementation developed using TDD"""
    
    def __init__(self):
        self._items = {}
        self._discount = 0.0
    
    def is_empty(self):
        """Check if cart is empty"""
        return len(self._items) == 0
    
    def total_items(self):
        """Get total number of items in cart"""
        return sum(item['quantity'] for item in self._items.values())
    
    def add_item(self, name, price, quantity):
        """Add item to cart"""
        if name in self._items:
            self._items[name]['quantity'] += quantity
        else:
            self._items[name] = {
                'price': price,
                'quantity': quantity
            }
    
    def remove_item(self, name, quantity):
        """Remove items from cart"""
        if name in self._items:
            if self._items[name]['quantity'] <= quantity:
                del self._items[name]
            else:
                self._items[name]['quantity'] -= quantity
    
    def get_items(self):
        """Get all items in cart"""
        return self._items.copy()
    
    def total_cost(self):
        """Calculate total cost with discount"""
        subtotal = sum(
            item['price'] * item['quantity'] 
            for item in self._items.values()
        )
        return subtotal * (1 - self._discount)
    
    def apply_discount(self, discount_rate):
        """Apply discount to cart"""
        self._discount = discount_rate

def demonstrate_tdd():
    """Demonstrate TDD process"""
    print("\nTest-Driven Development Demo:")
    print("=" * 31)
    
    print("TDD Process:")
    print("1. RED: Write a failing test")
    print("2. GREEN: Write minimal code to pass")
    print("3. REFACTOR: Improve code structure")
    print("4. Repeat")
    
    # Run the TDD tests
    print("\nRunning TDD Tests:")
    test_instance = TestShoppingCart()
    test_methods = [m for m in dir(test_instance) if m.startswith('test_')]
    
    passed = 0
    failed = 0
    
    for test_method in test_methods:
        try:
            test_instance.setup_method()
            getattr(test_instance, test_method)()
            print(f"✓ {test_method}")
            passed += 1
        except Exception as e:
            print(f"✗ {test_method}: {e}")
            failed += 1
    
    print(f"\nTDD Results: {passed} passed, {failed} failed")
    
    # Demonstrate the working shopping cart
    print("\nShopping Cart in Action:")
    cart = ShoppingCart()
    
    print("Adding items...")
    cart.add_item("Apple", 1.50, 3)
    cart.add_item("Banana", 0.75, 2)
    cart.add_item("Orange", 2.00, 1)
    
    print(f"Total items: {cart.total_items()}")
    print(f"Total cost: ${cart.total_cost():.2f}")
    
    print("Applying 15% discount...")
    cart.apply_discount(0.15)
    print(f"Discounted total: ${cart.total_cost():.2f}")

demonstrate_tdd()
```

---

## 6. Test Coverage and Quality Metrics

```python
"""
Test Coverage and Quality Metrics

Coverage Types:
1. Line Coverage: Percentage of code lines executed
2. Branch Coverage: Percentage of code branches taken
3. Function Coverage: Percentage of functions called
4. Statement Coverage: Percentage of statements executed
"""

class CoverageAnalyzer:
    """Simulate test coverage analysis"""
    
    def __init__(self):
        self.executed_lines = set()
        self.total_lines = 0
        self.executed_branches = set()
        self.total_branches = 0
        self.executed_functions = set()
        self.total_functions = 0
    
    def track_line(self, line_number):
        """Track executed line"""
        self.executed_lines.add(line_number)
    
    def track_branch(self, branch_id):
        """Track executed branch"""
        self.executed_branches.add(branch_id)
    
    def track_function(self, function_name):
        """Track executed function"""
        self.executed_functions.add(function_name)
    
    def set_totals(self, lines, branches, functions):
        """Set total counts"""
        self.total_lines = lines
        self.total_branches = branches
        self.total_functions = functions
    
    def get_coverage_report(self):
        """Generate coverage report"""
        line_coverage = (len(self.executed_lines) / self.total_lines * 100) if self.total_lines > 0 else 0
        branch_coverage = (len(self.executed_branches) / self.total_branches * 100) if self.total_branches > 0 else 0
        function_coverage = (len(self.executed_functions) / self.total_functions * 100) if self.total_functions > 0 else 0
        
        return {
            'line_coverage': line_coverage,
            'branch_coverage': branch_coverage,
            'function_coverage': function_coverage,
            'executed_lines': len(self.executed_lines),
            'total_lines': self.total_lines,
            'executed_branches': len(self.executed_branches),
            'total_branches': self.total_branches,
            'executed_functions': len(self.executed_functions),
            'total_functions': self.total_functions
        }

# Example class to analyze coverage
class MathUtils:
    """Math utility class for coverage demonstration"""
    
    def __init__(self):
        self.coverage = CoverageAnalyzer()
        self.coverage.set_totals(20, 8, 6)  # Simulated totals
    
    def add(self, a, b):
        """Add two numbers"""
        self.coverage.track_function('add')
        self.coverage.track_line(1)
        return a + b
    
    def subtract(self, a, b):
        """Subtract two numbers"""
        self.coverage.track_function('subtract')
        self.coverage.track_line(2)
        return a - b
    
    def multiply(self, a, b):
        """Multiply two numbers"""
        self.coverage.track_function('multiply')
        self.coverage.track_line(3)
        return a * b
    
    def divide(self, a, b):
        """Divide two numbers with error handling"""
        self.coverage.track_function('divide')
        self.coverage.track_line(4)
        
        if b == 0:
            self.coverage.track_branch('divide_zero')
            self.coverage.track_line(5)
            raise ValueError("Cannot divide by zero")
        else:
            self.coverage.track_branch('divide_normal')
            self.coverage.track_line(6)
            return a / b
    
    def factorial(self, n):
        """Calculate factorial"""
        self.coverage.track_function('factorial')
        self.coverage.track_line(7)
        
        if n < 0:
            self.coverage.track_branch('factorial_negative')
            self.coverage.track_line(8)
            raise ValueError("Factorial not defined for negative numbers")
        elif n == 0 or n == 1:
            self.coverage.track_branch('factorial_base')
            self.coverage.track_line(9)
            return 1
        else:
            self.coverage.track_branch('factorial_recursive')
            self.coverage.track_line(10)
            result = 1
            for i in range(2, n + 1):
                self.coverage.track_line(11)
                result *= i
            return result
    
    def is_prime(self, n):
        """Check if number is prime"""
        self.coverage.track_function('is_prime')
        self.coverage.track_line(12)
        
        if n < 2:
            self.coverage.track_branch('prime_less_than_2')
            self.coverage.track_line(13)
            return False
        
        self.coverage.track_line(14)
        for i in range(2, int(n ** 0.5) + 1):
            self.coverage.track_line(15)
            if n % i == 0:
                self.coverage.track_branch('prime_divisible')
                self.coverage.track_line(16)
                return False
        
        self.coverage.track_branch('prime_true')
        self.coverage.track_line(17)
        return True

class TestMathUtilsCoverage:
    """Test class that demonstrates coverage analysis"""
    
    def setup_method(self):
        """Set up test fixtures"""
        self.math_utils = MathUtils()
    
    def test_basic_operations(self):
        """Test basic arithmetic operations"""
        assert self.math_utils.add(2, 3) == 5
        assert self.math_utils.subtract(5, 3) == 2
        assert self.math_utils.multiply(3, 4) == 12
    
    def test_division_normal_case(self):
        """Test normal division"""
        assert self.math_utils.divide(10, 2) == 5.0
    
    def test_factorial_base_cases(self):
        """Test factorial base cases"""
        assert self.math_utils.factorial(0) == 1
        assert self.math_utils.factorial(1) == 1
        assert self.math_utils.factorial(5) == 120
    
    def test_prime_numbers(self):
        """Test prime number detection"""
        assert self.math_utils.is_prime(2) is True
        assert self.math_utils.is_prime(17) is True
        assert self.math_utils.is_prime(4) is False

def demonstrate_coverage():
    """Demonstrate coverage analysis"""
    print("\nTest Coverage Analysis:")
    print("=" * 25)
    
    # Run tests and collect coverage
    test_instance = TestMathUtilsCoverage()
    test_methods = [m for m in dir(test_instance) if m.startswith('test_')]
    
    print("Running tests with coverage tracking...")
    
    for test_method in test_methods:
        test_instance.setup_method()
        getattr(test_instance, test_method)()
        print(f"✓ {test_method}")
    
    # Generate coverage report
    coverage_report = test_instance.math_utils.coverage.get_coverage_report()
    
    print("\nCoverage Report:")
    print("-" * 16)
    print(f"Line Coverage:     {coverage_report['line_coverage']:.1f}% ({coverage_report['executed_lines']}/{coverage_report['total_lines']} lines)")
    print(f"Branch Coverage:   {coverage_report['branch_coverage']:.1f}% ({coverage_report['executed_branches']}/{coverage_report['total_branches']} branches)")
    print(f"Function Coverage: {coverage_report['function_coverage']:.1f}% ({coverage_report['executed_functions']}/{coverage_report['total_functions']} functions)")
    
    # Identify missing coverage
    print("\nCoverage Analysis:")
    if coverage_report['line_coverage'] < 100:
        print("⚠️  Not all lines are covered by tests")
        print("   Consider adding tests for edge cases")
    
    if coverage_report['branch_coverage'] < 100:
        print("⚠️  Not all code branches are covered")
        print("   Missing test cases:")
        print("   - Division by zero error case")
        print("   - Negative factorial error case")
    
    if coverage_report['function_coverage'] < 100:
        print("⚠️  Not all functions are tested")
    
    print("\nCoverage Improvement Suggestions:")
    print("1. Add test for divide by zero error")
    print("2. Add test for negative factorial input")
    print("3. Add more edge cases for prime number testing")
    print("4. Test prime numbers less than 2")

demonstrate_coverage()
```

---

## 7. Performance Testing

```python
import time
import random
from collections import defaultdict

class PerformanceTimer:
    """Simple performance timing utility"""
    
    def __init__(self):
        self.start_time = None
        self.measurements = defaultdict(list)
    
    def start(self):
        """Start timing"""
        self.start_time = time.time()
    
    def stop(self, operation_name="operation"):
        """Stop timing and record measurement"""
        if self.start_time is None:
            raise ValueError("Timer not started")
        
        elapsed = time.time() - self.start_time
        self.measurements[operation_name].append(elapsed)
        self.start_time = None
        return elapsed
    
    def get_stats(self, operation_name):
        """Get statistics for an operation"""
        measurements = self.measurements[operation_name]
        if not measurements:
            return None
        
        return {
            'count': len(measurements),
            'total': sum(measurements),
            'average': sum(measurements) / len(measurements),
            'min': min(measurements),
            'max': max(measurements)
        }

class DataProcessor:
    """Sample data processor for performance testing"""
    
    def __init__(self):
        self.data = []
    
    def add_data(self, items):
        """Add data items"""
        self.data.extend(items)
    
    def process_linear_search(self, target):
        """Linear search - O(n) complexity"""
        for i, item in enumerate(self.data):
            if item == target:
                return i
        return -1
    
    def process_sort_data(self):
        """Sort data - O(n log n) complexity"""
        return sorted(self.data)
    
    def process_filter_data(self, condition):
        """Filter data based on condition"""
        return [item for item in self.data if condition(item)]
    
    def process_aggregate_data(self):
        """Aggregate data - calculate statistics"""
        if not self.data:
            return {}
        
        return {
            'count': len(self.data),
            'sum': sum(self.data),
            'average': sum(self.data) / len(self.data),
            'min': min(self.data),
            'max': max(self.data)
        }

class PerformanceTestSuite:
    """Performance test suite"""
    
    def __init__(self):
        self.timer = PerformanceTimer()
        self.processor = DataProcessor()
    
    def setup_test_data(self, size):
        """Set up test data of specified size"""
        print(f"Setting up test data (size: {size:,})...")
        test_data = [random.randint(1, 1000) for _ in range(size)]
        self.processor.data = test_data
    
    def test_linear_search_performance(self, iterations=100):
        """Test linear search performance"""
        print("Testing linear search performance...")
        
        for i in range(iterations):
            target = random.choice(self.processor.data) if self.processor.data else 1
            
            self.timer.start()
            self.processor.process_linear_search(target)
            self.timer.stop("linear_search")
        
        stats = self.timer.get_stats("linear_search")
        print(f"  Linear Search - Avg: {stats['average']*1000:.2f}ms, Min: {stats['min']*1000:.2f}ms, Max: {stats['max']*1000:.2f}ms")
        return stats
    
    def test_sort_performance(self, iterations=10):
        """Test sorting performance"""
        print("Testing sort performance...")
        
        for i in range(iterations):
            # Create a copy to sort each time
            data_copy = self.processor.data.copy()
            
            self.timer.start()
            sorted(data_copy)
            self.timer.stop("sort")
        
        stats = self.timer.get_stats("sort")
        print(f"  Sort - Avg: {stats['average']*1000:.2f}ms, Min: {stats['min']*1000:.2f}ms, Max: {stats['max']*1000:.2f}ms")
        return stats
    
    def test_filter_performance(self, iterations=50):
        """Test filter performance"""
        print("Testing filter performance...")
        
        def is_even(x):
            return x % 2 == 0
        
        for i in range(iterations):
            self.timer.start()
            self.processor.process_filter_data(is_even)
            self.timer.stop("filter")
        
        stats = self.timer.get_stats("filter")
        print(f"  Filter - Avg: {stats['average']*1000:.2f}ms, Min: {stats['min']*1000:.2f}ms, Max: {stats['max']*1000:.2f}ms")
        return stats
    
    def test_aggregate_performance(self, iterations=100):
        """Test aggregation performance"""
        print("Testing aggregation performance...")
        
        for i in range(iterations):
            self.timer.start()
            self.processor.process_aggregate_data()
            self.timer.stop("aggregate")
        
        stats = self.timer.get_stats("aggregate")
        print(f"  Aggregate - Avg: {stats['average']*1000:.2f}ms, Min: {stats['min']*1000:.2f}ms, Max: {stats['max']*1000:.2f}ms")
        return stats
    
    def run_performance_comparison(self):
        """Run performance comparison across different data sizes"""
        print("\nPerformance Comparison Across Data Sizes:")
        print("=" * 45)
        
        data_sizes = [1000, 5000, 10000]
        results = {}
        
        for size in data_sizes:
            print(f"\n--- Data Size: {size:,} ---")
            self.setup_test_data(size)
            
            results[size] = {
                'linear_search': self.test_linear_search_performance(50),
                'sort': self.test_sort_performance(5),
                'filter': self.test_filter_performance(20),
                'aggregate': self.test_aggregate_performance(50)
            }
        
        # Performance analysis
        print("\nPerformance Analysis:")
        print("-" * 20)
        
        for operation in ['linear_search', 'sort', 'filter', 'aggregate']:
            print(f"\n{operation.replace('_', ' ').title()}:")
            for size in data_sizes:
                avg_time = results[size][operation]['average'] * 1000
                print(f"  {size:,} items: {avg_time:.2f}ms")

def demonstrate_performance_testing():
    """Demonstrate performance testing concepts"""
    print("Performance Testing Demo:")
    print("=" * 26)
    
    print("Performance Testing Types:")
    print("1. Load Testing: Normal expected load")
    print("2. Stress Testing: Beyond normal capacity")
    print("3. Spike Testing: Sudden load increases")
    print("4. Volume Testing: Large amounts of data")
    print("5. Endurance Testing: Extended periods")
    
    # Run performance tests
    perf_suite = PerformanceTestSuite()
    perf_suite.run_performance_comparison()
    
    print("\nPerformance Testing Best Practices:")
    print("1. Test with realistic data volumes")
    print("2. Test under various load conditions")
    print("3. Monitor memory usage and CPU")
    print("4. Set performance benchmarks")
    print("5. Test in production-like environment")
    print("6. Use profiling tools for bottlenecks")

demonstrate_performance_testing()
```

---

## 8. Testing Best Practices

```python
"""
Comprehensive Testing Best Practices
"""

def testing_best_practices_guide():
    """Comprehensive guide to testing best practices"""
    
    print("Testing Best Practices Guide:")
    print("=" * 30)
    
    practices = {
        "Test Organization": [
            "Use clear, descriptive test names",
            "Group related tests in test classes",
            "Follow consistent naming conventions",
            "Organize tests by feature or component",
            "Separate unit, integration, and e2e tests"
        ],
        
        "Test Structure": [
            "Follow AAA pattern (Arrange, Act, Assert)",
            "Keep tests focused on single behavior",
            "Make tests independent and isolated",
            "Use setup/teardown methods appropriately",
            "Avoid complex logic in tests"
        ],
        
        "Test Data Management": [
            "Use fixtures for common test data",
            "Create realistic but minimal test data",
            "Avoid hardcoded values when possible",
            "Use factories for complex object creation",
            "Clean up test data after tests"
        ],
        
        "Assertions": [
            "Use specific, meaningful assertions",
            "Include helpful error messages",
            "Test both positive and negative cases",
            "Verify all important outcomes",
            "Use custom matchers when helpful"
        ],
        
        "Mocking Strategy": [
            "Mock external dependencies",
            "Don't mock what you don't own",
            "Verify interactions when important",
            "Use test doubles appropriately",
            "Keep mocks simple and focused"
        ],
        
        "Test Maintenance": [
            "Refactor tests as code evolves",
            "Remove obsolete tests promptly",
            "Keep tests DRY but readable",
            "Update tests when requirements change",
            "Monitor test execution time"
        ]
    }
    
    for category, items in practices.items():
        print(f"\n{category}:")
        for item in items:
            print(f"  • {item}")

def demonstrate_good_vs_bad_tests():
    """Show examples of good vs bad test practices"""
    
    print("\n\nGood vs Bad Test Examples:")
    print("=" * 28)
    
    print("\n❌ BAD TEST EXAMPLE:")
    print("""
def test_stuff():  # Vague name
    # No arrange section
    result = some_function(1, 2, "test", True, None)  # Unclear parameters
    assert result  # Vague assertion
    
    # Multiple unrelated assertions
    assert other_function() == "something"
    assert yet_another_function() is not None
    """)
    
    print("\n✅ GOOD TEST EXAMPLE:")
    print("""
def test_calculate_discount_with_valid_customer_returns_correct_amount():
    # Arrange
    customer = Customer(membership_level="premium")
    order = Order(subtotal=100.00)
    discount_calculator = DiscountCalculator()
    
    # Act
    discount_amount = discount_calculator.calculate_discount(customer, order)
    
    # Assert
    expected_discount = 10.00  # 10% premium discount
    assert discount_amount == expected_discount, f"Expected {expected_discount}, got {discount_amount}"
    """)
    
    print("\nWhy the good test is better:")
    print("• Descriptive name explains what's being tested")
    print("• Clear AAA structure")
    print("• Meaningful variable names")
    print("• Specific assertion with helpful message")
    print("• Tests only one behavior")

class TestQualityChecker:
    """Tool to check test quality"""
    
    def __init__(self):
        self.quality_metrics = {
            'descriptive_names': 0,
            'aaa_structure': 0,
            'single_assertion': 0,
            'meaningful_data': 0,
            'clear_assertions': 0
        }
    
    def check_test_name(self, test_name):
        """Check if test name is descriptive"""
        if len(test_name) > 20 and '_' in test_name and any(
            word in test_name.lower() for word in 
            ['test', 'should', 'when', 'given', 'returns', 'raises']
        ):
            self.quality_metrics['descriptive_names'] += 1
            return True
        return False
    
    def check_aaa_structure(self, test_code):
        """Check if test follows AAA structure"""
        # Simplified check for comments or structure
        if '# arrange' in test_code.lower() or '# act' in test_code.lower():
            self.quality_metrics['aaa_structure'] += 1
            return True
        return False
    
    def generate_quality_report(self, total_tests):
        """Generate test quality report"""
        print("\nTest Quality Report:")
        print("-" * 19)
        
        for metric, count in self.quality_metrics.items():
            percentage = (count / total_tests * 100) if total_tests > 0 else 0
            metric_name = metric.replace('_', ' ').title()
            print(f"{metric_name}: {percentage:.1f}% ({count}/{total_tests})")
        
        overall_score = sum(self.quality_metrics.values()) / (len(self.quality_metrics) * total_tests) * 100 if total_tests > 0 else 0
        print(f"\nOverall Quality Score: {overall_score:.1f}%")
        
        if overall_score >= 80:
            print("🎉 Excellent test quality!")
        elif overall_score >= 60:
            print("👍 Good test quality, room for improvement")
        else:
            print("⚠️  Test quality needs improvement")

def demonstrate_test_quality_checking():
    """Demonstrate test quality checking"""
    
    print("\n\nTest Quality Assessment:")
    print("=" * 25)
    
    checker = TestQualityChecker()
    
    # Sample test names to check
    test_names = [
        "test_user_registration_with_valid_email_creates_user_account",
        "test_login",
        "test_calculate_tax_with_zero_income_returns_zero",
        "test_stuff",
        "test_order_processing_with_expired_coupon_raises_validation_error"
    ]
    
    print("Checking test name quality...")
    for name in test_names:
        is_good = checker.check_test_name(name)
        status = "✅" if is_good else "❌"
        print(f"{status} {name}")
    
    # Sample test code structures
    test_codes = [
        "# arrange\nuser = User()\n# act\nresult = user.login()\n# assert\nassert result",
        "result = function(); assert result == True",
        "# arrange\ndata = setup_data()\n# act\noutput = process(data)\n# assert\nassert output.status == 'success'"
    ]
    
    print("\nChecking AAA structure...")
    for i, code in enumerate(test_codes):
        is_good = checker.check_aaa_structure(code)
        status = "✅" if is_good else "❌"
        print(f"{status} Test {i+1}: {'Follows AAA' if is_good else 'Missing AAA structure'}")
    
    checker.generate_quality_report(len(test_names))

# Run all demonstrations
testing_best_practices_guide()
demonstrate_good_vs_bad_tests()
demonstrate_test_quality_checking()
```

---

## 9. Continuous Integration Testing

```python
"""
Continuous Integration (CI) Testing Concepts and Configuration
"""

class CITestRunner:
    """Simulated CI test runner"""
    
    def __init__(self):
        self.test_results = {
            'unit_tests': {'passed': 0, 'failed': 0, 'skipped': 0},
            'integration_tests': {'passed': 0, 'failed': 0, 'skipped': 0},
            'e2e_tests': {'passed': 0, 'failed': 0, 'skipped': 0},
            'lint_checks': {'passed': 0, 'failed': 0, 'skipped': 0},
            'security_scans': {'passed': 0, 'failed': 0, 'skipped': 0}
        }
        self.build_status = 'pending'
    
    def run_test_suite(self, suite_name, test_count):
        """Simulate running a test suite"""
        import random
        
        print(f"Running {suite_name}...")
        
        # Simulate test execution
        for i in range(test_count):
            outcome = random.choices(
                ['passed', 'failed', 'skipped'], 
                weights=[85, 10, 5]  # 85% pass rate
            )[0]
            self.test_results[suite_name][outcome] += 1
        
        suite_results = self.test_results[suite_name]
        total = sum(suite_results.values())
        print(f"  {suite_results['passed']}/{total} passed, {suite_results['failed']} failed, {suite_results['skipped']} skipped")
        
        return suite_results['failed'] == 0
    
    def run_full_pipeline(self):
        """Run complete CI pipeline"""
        print("CI Pipeline Execution:")
        print("=" * 21)
        
        pipeline_stages = [
            ('unit_tests', 150),
            ('integration_tests', 25),
            ('lint_checks', 1),
            ('security_scans', 1),
            ('e2e_tests', 10)
        ]
        
        all_passed = True
        
        for stage_name, test_count in pipeline_stages:
            stage_passed = self.run_test_suite(stage_name, test_count)
            if not stage_passed:
                all_passed = False
                print(f"❌ {stage_name} failed - stopping pipeline")
                break
            else:
                print(f"✅ {stage_name} passed")
        
        if all_passed:
            self.build_status = 'success'
            print("\n🎉 All tests passed! Build is ready for deployment.")
        else:
            self.build_status = 'failed'
            print("\n❌ Build failed. Check test results above.")
        
        return all_passed
    
    def generate_test_report(self):
        """Generate detailed test report"""
        print("\nDetailed Test Report:")
        print("-" * 21)
        
        total_tests = 0
        total_passed = 0
        total_failed = 0
        total_skipped = 0
        
        for suite_name, results in self.test_results.items():
            suite_total = sum(results.values())
            if suite_total > 0:
                pass_rate = (results['passed'] / suite_total) * 100
                print(f"{suite_name.replace('_', ' ').title()}:")
                print(f"  Total: {suite_total}, Passed: {results['passed']}, Failed: {results['failed']}, Skipped: {results['skipped']}")
                print(f"  Pass Rate: {pass_rate:.1f}%")
                
                total_tests += suite_total
                total_passed += results['passed']
                total_failed += results['failed']
                total_skipped += results['skipped']
        
        if total_tests > 0:
            overall_pass_rate = (total_passed / total_tests) * 100
            print(f"\nOverall Results:")
            print(f"  Total Tests: {total_tests}")
            print(f"  Passed: {total_passed} ({(total_passed/total_tests)*100:.1f}%)")
            print(f"  Failed: {total_failed} ({(total_failed/total_tests)*100:.1f}%)")
            print(f"  Skipped: {total_skipped} ({(total_skipped/total_tests)*100:.1f}%)")
            print(f"  Overall Pass Rate: {overall_pass_rate:.1f}%")

def demonstrate_ci_configuration():
    """Show CI configuration examples"""
    
    print("CI Configuration Examples:")
    print("=" * 28)
    
    print("\n1. GitHub Actions Workflow (.github/workflows/test.yml):")
    print("""
name: Test Suite

on: [push, pull_request]

jobs:
  test:
    runs-on: ubuntu-latest
    strategy:
      matrix:
        python-version: [3.8, 3.9, '3.10', '3.11']
    
    steps:
    - uses: actions/checkout@v3
    - name: Set up Python ${{ matrix.python-version }}
      uses: actions/setup-python@v3
      with:
        python-version: ${{ matrix.python-version }}
    
    - name: Install dependencies
      run: |
        python -m pip install --upgrade pip
        pip install -r requirements.txt
        pip install pytest pytest-cov
    
    - name: Run tests with coverage
      run: |
        pytest --cov=src --cov-report=xml
    
    - name: Upload coverage reports
      uses: codecov/codecov-action@v3
    """)
    
    print("\n2. pytest Configuration (pytest.ini):")
    print("""
[tool:pytest]
testpaths = tests
python_files = test_*.py
python_classes = Test*
python_functions = test_*
addopts = 
    --strict-markers
    --strict-config
    --verbose
    --tb=short
    --cov=src
    --cov-branch
    --cov-report=term-missing
    --cov-report=html
    --cov-fail-under=80

markers =
    slow: marks tests as slow
    integration: marks tests as integration tests
    unit: marks tests as unit tests
    """)
    
    print("\n3. CI Best Practices:")
    practices = [
        "Run tests on multiple Python versions",
        "Fail fast on test failures",
        "Generate and store test reports",
        "Check code coverage thresholds",
        "Run linting and security checks",
        "Cache dependencies for faster builds",
        "Run tests in parallel when possible",
        "Use matrix builds for different environments"
    ]
    
    for practice in practices:
        print(f"  • {practice}")

def demonstrate_ci_pipeline():
    """Demonstrate CI pipeline execution"""
    
    print("\n\nCI Pipeline Demonstration:")
    print("=" * 28)
    
    # Run CI pipeline
    ci_runner = CITestRunner()
    success = ci_runner.run_full_pipeline()
    
    # Generate detailed report
    ci_runner.generate_test_report()
    
    print(f"\nBuild Status: {ci_runner.build_status.upper()}")
    
    if success:
        print("\nNext Steps:")
        print("• Deploy to staging environment")
        print("• Run smoke tests in staging")
        print("• Deploy to production if staging tests pass")
    else:
        print("\nRequired Actions:")
        print("• Fix failing tests")
        print("• Address any security vulnerabilities")
        print("• Re-run CI pipeline")

# Run CI demonstrations
demonstrate_ci_configuration()
demonstrate_ci_pipeline()
```

---

## 10. Summary and Final Exercise

```python
"""
Comprehensive Testing Workshop - Final Exercise

This exercise combines all the testing concepts learned:
- Unit testing with pytest
- Mocking and test doubles
- Integration testing
- TDD approach
- Coverage analysis
- Performance considerations
"""

print("🎓 Testing Strategies Workshop Complete!")
print("=" * 45)

print("\nKey Concepts Covered:")
concepts = [
    "Testing Pyramid: Unit → Integration → E2E",
    "pytest framework and fixtures",
    "Mock objects and test doubles",
    "Test-driven development (TDD)",
    "Code coverage measurement",
    "Performance and load testing",
    "Continuous integration setup",
    "Testing best practices and patterns"
]

for i, concept in enumerate(concepts, 1):
    print(f"{i}. {concept}")

print("\n📚 Further Learning Resources:")
resources = [
    "Official pytest documentation: https://docs.pytest.org/",
    "Test-driven development with Python book",
    "Clean Code and clean tests principles",
    "Mock library documentation for advanced mocking",
    "Performance testing with locust or siege",
    "CI/CD pipeline design patterns",
    "Testing in microservices architectures"
]

for resource in resources:
    print(f"• {resource}")

print("\n🛠️ Practical Next Steps:")
next_steps = [
    "Set up pytest in your current project",
    "Write unit tests for core business logic",
    "Implement integration tests for database/API layers",
    "Set up CI pipeline with automated testing",
    "Measure and improve test coverage",
    "Practice TDD on new features",
    "Add performance tests for critical paths"
]

for i, step in enumerate(next_steps, 1):
    print(f"{i}. {step}")

print("\n🏆 Final Challenge:")
print("Build a complete test suite for a real project including:")
print("• Unit tests with >90% coverage")
print("• Integration tests for external dependencies")
print("• Performance benchmarks")
print("• CI pipeline configuration")
print("• Documentation of testing strategy")

print("\nHappy Testing! 🧪✨")
```

---

## Notebook Complete! 

This comprehensive testing notebook covers all essential testing strategies and practices in Python. Students can work through each section to build their testing expertise progressively, from basic unit tests to advanced CI/CD integration.

**Practice Exercises:**
1. Implement the Calculator tests with pytest
2. Build a complete test suite for the Shopping Cart using TDD
3. Create integration tests for the UserRepository
4. Set up a CI pipeline for your project
5. Measure and improve test coverage for existing code

Remember: Good tests are the foundation of maintainable, reliable software!